In [44]:
# ドライブ読み込み
from google.colab import drive
drive.mount('/content/drive')
%cd "/content/drive/My Drive/Colab Notebooks"

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/My Drive/Colab Notebooks


In [ ]:
#!pip install japanize_matplotlib

In [45]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
#import japanize_matplotlib
import seaborn as sns
import sympy
import math
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from PIL import Image
import glob
import os
import re

# ↓↓↓↓↓↓---------ここから画像データの数値化と保存の作業部分---------↓↓↓↓↓↓

特にいじるところない

In [46]:
# ファイル名を指定した画像を NumPy配列 にして返す関数
def img2array(img_path):
    img_c1 = Image.open(img_path).convert("L")
#    plt.imshow(img_c1, cmap="gray")
#    plt.show()
    img_c2 = img_c1.resize((8,8),Image.LANCZOS)
#    plt.imshow(img_c2, cmap="gray")
#    plt.show()
    img_arr1 = np.asarray(img_c2, dtype=float)
#    print(img_arr1)
    img_arr2 = 16 - np.floor(17 * img_arr1/256)
#    print(img_arr2)
#    plt.imshow(img_arr2, cmap="gray")
#    plt.show()
    img_arr3 = img_arr2.reshape(1,-1)
#    print(img_arr3.shape)
    return img_arr3

In [47]:
def imgs2csv(img_list, csv_name):
    csv_array = np.empty((0, 64+1),float)
    for img_path in img_list:
        #print(img2array(img_path))
        ans_label = -1
        if re.search('_maru', img_path):
            ans_label = 0
        elif re.search('_batsu', img_path):
            ans_label = 1
        elif re.search('_sankaku', img_path):
            ans_label = 2
        elif re.search('_shikaku', img_path):
            ans_label = 3

        img_arr_x = img2array(img_path)
        img_arr_xy = np.append(img_arr_x, np.array([[ans_label]]), axis=1)

        csv_array = np.append(csv_array, img_arr_xy,  axis=0)

    h_str = ",".join(["x"+str(i) for i in range(1,65)]) + ",y(label)"
    np.savetxt(csv_name, csv_array, delimiter=",", comments="", header=h_str)
    return csv_array

In [48]:
def folder2csv(dir_path, csv_name, ex_list):
    file_list = glob.glob(dir_path + '*.*')
    # dir_path + '*.*' と打つと、拡張子があるファイルのみがリスト化される。
    img_list = []

    for file_name in file_list:
        name, ext = os.path.splitext(file_name)
        #print(ext)
        if ext in ex_list:
            img_list.append(file_name)
    print("あなたが指定したフォルダ内の画像リストは")
    print(img_list)
    img_array = imgs2csv(img_list, csv_name)
    return img_array

# ↑↑↑↑↑↑---------ここまで画像データの数値化と保存の作業部分---------↑↑↑↑↑↑

# ★★コード書き換えポイント★★

↓モデルに使う画像の前処理をするので、

「大量の画像が入ってるフォルダのパス」
「大量の画像をcsvファイルに書き込む際の名前(自由につける)」

を第一引数と第二引数に入れる

In [ ]:
# 大量の画像の数値化とCSV形式での保存、を実行
folder2csv("./hukusei_gazou5/", "gazou5_df.csv", ['.png'])

あなたが指定したフォルダ内の画像リストは
['./hukusei_gazou5/973008_maru__rotated_image_180.png', './hukusei_gazou5/973008_maru__rotated_image_200.png', './hukusei_gazou5/973008_maru__rotated_image_220.png', './hukusei_gazou5/973008_maru__rotated_image_240.png', './hukusei_gazou5/973008_maru__rotated_image_260.png', './hukusei_gazou5/973008_maru__rotated_image_280.png', './hukusei_gazou5/973008_maru__rotated_image_300.png', './hukusei_gazou5/973008_maru__rotated_image_320.png', './hukusei_gazou5/973008_maru__rotated_image_340.png', './hukusei_gazou5/973008_maru__rotated_image_360.png', './hukusei_gazou5/973134_shikaku__rotated_image_20.png', './hukusei_gazou5/973134_shikaku__rotated_image_40.png', './hukusei_gazou5/973134_shikaku__rotated_image_60.png', './hukusei_gazou5/973134_shikaku__rotated_image_80.png', './hukusei_gazou5/973134_shikaku__rotated_image_100.png', './hukusei_gazou5/973134_shikaku__rotated_image_120.png', './hukusei_gazou5/973134_shikaku__rotated_image_140.png', './hukusei_gazou5/973134_s

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

# ★★コード書き換えポイント★★

↓先ほど名前を付けたcsvファイルのパスをここに入れて読み込む

In [ ]:
# -----------------------------確かめ----------------------------
df = pd.read_csv("gazou5_df.csv")
#print(df)

# arr_x = df.iloc[:,0:64].to_numpy()
#arr_x = df.drop('y(label)', axis=1).to_numpy() #でも良い。
# for i in range(0, len(arr_x)):
#     plt.imshow(arr_x[i].reshape(8,8),cmap="gray")
#     plt.show()

# ↑↑↑↑↑↑---------ここまで先生が用意したコード---------↑↑↑↑↑↑

# ↓↓↓↓↓↓---------ここからモデルの選択と学習と予測---------↓↓↓↓↓↓

# 学習と正解率の計算(ここでモデルの選択と正確さを計る)

# ★★コード書き換えポイント★★

使いたいモデルのコメントアウトを外して試す

In [ ]:
# 説明変数(8x8の計64)
X = df.iloc[:,0:64]
# 目的変数(画像の名前を表す数値)
y = df["y(label)"]

# 画像のデータたちを学習データとテストデータに分ける
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)





# ↓↓↓↓↓↓-------------ここから学習モデル選択---------------↓↓↓↓↓↓


# 先生から頂いたものをそのまま貼り付けている。
# 試したいもののみを適宜コメントアウトから外して試していく

# from sklearn.linear_model import LogisticRegression
# model = LogisticRegression()

# from sklearn.svm import SVC
# model = SVC(probability=True)
# model = SVC(probability=True, C=1.3)
# model = SVC(probability=True,C=2.2,gamma=1.5)

from sklearn.ensemble import RandomForestClassifier
# model = RandomForestClassifier(n_estimators=300, max_depth=200)
model = RandomForestClassifier(n_estimators=300, max_depth=200)

# from sklearn.neighbors import KNeighborsClassifier
# model = KNeighborsClassifier(n_neighbors=6)

# from sklearn.neural_network import MLPClassifier
# model = MLPClassifier(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
#        beta_2=0.999, early_stopping=False, epsilon=1e-08,
#        hidden_layer_sizes=(30,30,),max_iter=10000)

# from sklearn.tree import DecisionTreeClassifier
# model = DecisionTreeClassifier()

# model = DecisionTreeClassifier(criterion='gini',               # 不純度の計測指標
#                                splitter='best',                # 各ノードの分割方法
#                                max_depth=None,                 # 決定木の深さの最大値
#                                min_samples_split=2,            # 枝の分岐作成に必要な最低データ数
#                                min_samples_leaf=1,             # 葉ノードの作成に必要な最低データ数
#                                min_weight_fraction_leaf=0.0,   # 各葉ノードに必要なデータの割合の最小値
#                                max_features=None,              # 分割時に利用する特徴数の最大値
#                                random_state=None,              # 乱数シード
#                                max_leaf_nodes=None,            # 作成する葉ノードの最大値
#                                min_impurity_decrease=0.0,      # ノードの分割実行するための不純度の最小減少度
#                                class_weight=None,              # 各クラスラベルに対する重み
#                                ccp_alpha=0.0                   # コスト複雑度枝刈りのパラメータ
#                                    )

# from sklearn.naive_bayes import MultinomialNB
# model = MultinomialNB()


# ↑↑↑↑↑↑-------------ここまで学習モデル選択---------------↑↑↑↑↑↑





# モデル学習
model.fit(X_train, y_train)

# モデルテスト
y_pred = model.predict(X_test)
y_train_pred = model.predict(X_train)
seikairitsu1 = accuracy_score(y_train, y_train_pred)
seikairitsu2 = accuracy_score(y_test, y_pred)
print('学習データの正解率は', seikairitsu1*100, "%")
print('テストデータの正解率は', seikairitsu2*100, "%")

学習データの正解率は 91.90863337204802 %
テストデータの正解率は 75.06775067750678 %




# モデルの書き込み保存(回答者用)

In [43]:
#from sklearn.externals import joblib
#以前は、joblibが sklearn.externals にバインドされていたので、上記のように書いていましたが、バージョンアップに伴い、joblibは格上げされました。下記で良いです。
import joblib
#学習済みモデルを保存する
joblib.dump(model, 'RFC_model6')

NameError: ignored

# ↓↓↓↓↓↓------------ここ試合本番の画像予測させる手順------------↓↓↓↓↓↓

In [58]:
# このセルは別のノートブックに！！

import joblib
#保存した学習済みモデルを呼び出す
RFC_model5 = joblib.load('RFC_model6')



推測する画像の読み込みとNumpyへの変換

(理由: モデルに予測させるため同じ説明変数の数のデータまで落とし込む必要がある)

↓

モデルは二次元配列しか受け取れないため、二次元へ変換

その後モデルへpredictして、結果をわかりやすくprint文へ

# ★★コード書き換えポイント★★

始めのopenメソッドで開くパスに、推測する1枚の画像のパスを入れる。

In [69]:
from PIL import Image


# 出題者から出題された画像がある場所のパス
test_img_path = "M5_5_maru_.png"

# ---------出題者からの画像をNumpy配列に変換します--------

# ① 画像ファイルを読み込み、② 8bitグレースケールモードに変換する。
img_c1a = Image.open(test_img_path).convert("L")
# ② 大きさを 8×8 にする。
img_c2a = img_c1a.resize((8,8))
# ③ 画像オブジェクトを NumPy 配列に変換する。
img_arr1a = np.asarray(img_c2a, dtype=float)
# ④ 256階調の値を白黒逆転させ 16 階調にする。
img_arr2a = 16 - np.floor(17 * img_arr1a/256)
# ⑤ NumPy配列の形状をフラットに変更する。
img_arr3a = img_arr2a.flatten()
# 二次元配列に変換
test_num = np.reshape(img_arr3a, (1, len(img_arr3a)))



# ------------推測部分--------------

# <外部からモデル読み込みしてる場合>予測させる(戻り値は数字(内容は〇×△□))
test_png_pred = RFC_model5.predict(test_num)

# <検証中のとき>予測させる(戻り値は数字(内容は〇×△□))
# test_png_pred = RFC_model5.predict(test_num)

# 予測の数値を図形の名前に変える
zukei_name = ""
if test_png_pred == 0:
  zukei_name = "〇(まる)"
elif test_png_pred == 1:
  zukei_name = "×(ばつ)"
elif test_png_pred == 2:
  zukei_name = "△(さんかく)"
elif test_png_pred == 3:
  zukei_name = "□(しかく)"

# 予測した図形の名前を表示
print(f'このモデルは{test_img_path}のことを{zukei_name}と予測しました。')

このモデルはM5_5_maru_.pngのことを×(ばつ)と予測しました。


/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
